In [1]:
import pandas as pd
import numpy as np
import lightgbm as lgb
from sklearn.model_selection import KFold, train_test_split
from sklearn.metrics import accuracy_score, f1_score, confusion_matrix, classification_report, roc_auc_score
import talib as ta


# 数据加载

In [2]:
# 原始数据
data_path = './data/pre/000001.SH.csv'
df = pd.read_csv(data_path, index_col=0)
df

,symbol,close,high,low,open,pre_close,change,pct_chg,volume,amount,label
date,,,,,,,,,,,
1990-12-20,'000001,104.3900,104.3900,99.9800,104.3000,99.9800,4.4100,4.4109,197.0,8.400000e+04,1
1990-12-21,'000001,109.1300,109.1300,103.7300,109.0700,104.3900,4.7400,4.5407,28.0,1.600000e+04,1
1990-12-24,'000001,114.5500,114.5500,109.1300,113.5700,109.1300,5.4200,4.9666,32.0,3.100000e+04,1
1990-12-25,'000001,120.2500,120.2500,114.5500,120.0900,114.5500,5.7000,4.9760,15.0,6.000000e+03,1
1990-12-26,'000001,125.2700,125.2700,120.2500,125.2700,120.2500,5.0200,4.1746,100.0,5.300000e+04,1
...,...,...,...,...,...,...,...,...,...,...,...
2021-01-14,'000001,3565.9046,3599.0584,3559.6001,3584.9324,3598.6518,-32.7472,-0.9100,347666819.0,5.113251e+11,1
2021-01-15,'000001,3566.3778,3589.2655,3533.7882,3566.2847,3565.9046,0.4732,0.0133,324612744.0,4.803438e+11,1
2021-01-18,'000001,3596.2237,3608.7665,3544.2614,3554.7977,3566.3778,29.8459,0.8369,301652565.0,4.517407e+10,0


# 加入特征

In [3]:
df['ADX'] = ta.ADX(df['high'], df['low'], df['close'], timeperiod=14)
df['ADXR'] = ta.ADXR(df['high'], df['low'], df['close'], timeperiod=14)
df['AROONDOWN'], df['AROONUP'] = ta.AROON(df['high'], df['low'], timeperiod=14)
df['AROONOSC'] = ta.AROONOSC(df['high'], df['low'], timeperiod=14)
df['ULTOSC'] = ta.ULTOSC(df['high'], df['low'], df['close'], timeperiod1=7, timeperiod2=14, timeperiod3=28)
df['MA5'] = ta.MA(df['close'], timeperiod=5, matype=0)
df['MA10'] = ta.MA(df['close'], timeperiod=10, matype=0)
df['MA20'] = ta.MA(df['close'], timeperiod=20, matype=0)
df['MA30'] = ta.MA(df['close'], timeperiod=30, matype=0)
df['MA60'] = ta.MA(df['close'], timeperiod=60, matype=0)

df['K'] , df['D']= ta.STOCHF(df['high'], df['low'], df['close'], fastk_period=5, fastd_period=3, fastd_matype=0)

df['DIF'], df['DEM'], df['HISTOGRAM'] = ta.MACD(df['close'], fastperiod=12, slowperiod=26, signalperiod=9)

# 布林线
df['UPPERBAND'], df['MIDDLEBAND'], df['LOWERBAND'] = ta.BBANDS(df['close'], timeperiod=5, nbdevup=2, nbdevdn=2, matype=0)

df['RSI6'] = ta.RSI(df['close'], timeperiod=6)
df['RSI12'] = ta.RSI(df['close'], timeperiod=12)
df['RSI24'] = ta.RSI(df['close'], timeperiod=24)

df['EMA'] = ta.EMA(df['close'], timeperiod=10)
df['NATR'] = ta.NATR(df['high'], df['low'], df['close'], timeperiod=14)
df['CDL3OUTSIDE'] = ta.CDL3OUTSIDE(df['open'], df['high'], df['low'], df['close'])
df['TRANGE'] = ta.TRANGE(df['high'], df['low'], df['close'])

# 形态模式
df['CDLBELTHOLD'] = ta.CDLBELTHOLD(df['open'], df['high'], df['low'], df['close'])
df['CDLCLOSINGMARUBOZU'] = ta.CDLCLOSINGMARUBOZU(df['open'], df['high'], df['low'], df['close'])


df

,symbol,close,high,low,open,pre_close,change,pct_chg,volume,amount,...,LOWERBAND,RSI6,RSI12,RSI24,EMA,NATR,CDL3OUTSIDE,TRANGE,CDLBELTHOLD,CDLCLOSINGMARUBOZU
date,,,,,,,,,,,,,,,,,,,,,
1990-12-20,'000001,104.3900,104.3900,99.9800,104.3000,99.9800,4.4100,4.4109,197.0,8.400000e+04,...,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,0,0
1990-12-21,'000001,109.1300,109.1300,103.7300,109.0700,104.3900,4.7400,4.5407,28.0,1.600000e+04,...,NaN,NaN,NaN,NaN,NaN,NaN,0,5.4000,0,0
1990-12-24,'000001,114.5500,114.5500,109.1300,113.5700,109.1300,5.4200,4.9666,32.0,3.100000e+04,...,NaN,NaN,NaN,NaN,NaN,NaN,0,5.4200,0,0
1990-12-25,'000001,120.2500,120.2500,114.5500,120.0900,114.5500,5.7000,4.9760,15.0,6.000000e+03,...,NaN,NaN,NaN,NaN,NaN,NaN,0,5.7000,0,0
1990-12-26,'000001,125.2700,125.2700,120.2500,125.2700,120.2500,5.0200,4.1746,100.0,5.300000e+04,...,99.754864,NaN,NaN,NaN,NaN,NaN,0,5.0200,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-01-14,'000001,3565.9046,3599.0584,3559.6001,3584.9324,3598.6518,-32.7472,-0.9100,347666819.0,5.113251e+11,...,3520.715852,59.453545,62.935210,61.295314,3542.416154,1.338962,0,39.4583,0,0
2021-01-15,'000001,3566.3778,3589.2655,3533.7882,3566.2847,3565.9046,0.4732,0.0133,324612744.0,4.803438e+11,...,3519.624754,59.577316,62.991694,61.326221,3546.772817,1.354268,0,55.4773,0,0
2021-01-18,'000001,3596.2237,3608.7665,3544.2614,3554.7977,3566.3778,29.8459,0.8369,301652565.0,4.517407e+10,...,3551.926286,67.163764,66.503946,63.257251,3555.763887,1.375219,0,64.5051,0,0


In [4]:
# 数据集划分
train_start_time = '2016-01-01'
train_end_time = '2020-11-30'
valid_start_time = '2021-01-01'
valid_end_time = '2021-01-07'
test_start_time = '2020-12-01'
test_end_time = '2021-01-20'
train_df = df.loc[train_start_time:train_end_time]
valid_df = df.loc[valid_start_time:valid_end_time]
test_df = df.loc[test_start_time:test_end_time]
train_df

,symbol,close,high,low,open,pre_close,change,pct_chg,volume,amount,...,LOWERBAND,RSI6,RSI12,RSI24,EMA,NATR,CDL3OUTSIDE,TRANGE,CDLBELTHOLD,CDLCLOSINGMARUBOZU
date,,,,,,,,,,,,,,,,,,,,,
2016-01-04,'000001,3296.2580,3538.6890,3295.7410,3536.5890,3539.1820,-242.9240,-6.8638,184418423.0,2.409292e+11,...,3294.181289,15.963421,28.672971,39.698774,3521.130646,2.276708,0,243.4410,-100,-100
2016-01-05,'000001,3287.7110,3328.1390,3189.6050,3196.6510,3296.2580,-8.5470,-0.2593,266882083.0,3.284172e+11,...,3189.741880,15.567306,28.244298,39.389240,3478.690711,2.420560,0,138.5340,100,0
2016-01-06,'000001,3361.8400,3362.9740,3288.9330,3291.1950,3287.7110,74.1290,2.2547,238886670.0,2.852432e+11,...,3169.225831,32.897056,37.136602,43.384316,3457.445127,2.358012,0,75.2630,100,100
2016-01-07,'000001,3125.0020,3309.6570,3115.8850,3309.6570,3361.8400,-236.8380,-7.0449,70569123.0,7.998199e+10,...,3054.563606,18.410004,25.934715,35.568306,3397.000922,2.917709,0,245.9550,-100,0
2016-01-08,'000001,3186.4120,3235.4510,3056.8780,3194.6250,3125.0020,61.4100,1.9651,286440822.0,3.241671e+11,...,3082.382228,28.242410,31.757370,38.563006,3358.712027,3.057387,0,178.5730,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2020-11-24,'000001,3402.8225,3413.9263,3396.2436,3407.4087,3414.4899,-11.6674,-0.3417,300994144.0,3.595058e+11,...,3331.570064,70.947945,64.554994,58.410928,3365.134619,1.105950,0,18.2463,0,0
2020-11-25,'000001,3362.3274,3423.4853,3362.3274,3417.5150,3402.8225,-40.4951,-1.1900,322110207.0,3.834778e+11,...,3341.850899,50.137992,54.712472,54.128854,3364.624216,1.169245,0,61.1579,0,-100
2020-11-26,'000001,3369.7334,3371.4537,3344.2822,3360.0609,3362.3274,7.4060,0.2203,255886822.0,3.173086e+11,...,3345.555905,53.153568,56.049409,54.761747,3365.553158,1.140937,0,27.1715,0,0


In [5]:
# 选择特征
all_features = ['close', 'high', 'low', 'open', 'pre_close', 'change', 'pct_chg', 'volume', 'amount', 'ADX', 'ADXR', 'AROONDOWN', 'AROONUP', 'AROONOSC', 'ULTOSC', 'MA5', 'MA10', 'MA20', 'MA30', 'MA60', 'K', 'D', 'DIF', 'DEM', 'HISTOGRAM', 'UPPERBAND', 'MIDDLEBAND', 'LOWERBAND', 'RSI6', 'RSI12', 'RSI24', 'EMA', 'NATR', 'CDL3OUTSIDE', 'TRANGE', 'CDLBELTHOLD', 'CDLCLOSINGMARUBOZU']

selected_features = ['close', 'high', 'low', 'open', 'pre_close', 'change', 'pct_chg', 'amount', 'ADX', 'ADXR', 'AROONDOWN', 'AROONOSC', 'ULTOSC', 'MA5', 'MA10', 'MA20', 'MA30', 'MA60', 'K', 'D', 'DEM', 'HISTOGRAM', 'UPPERBAND', 'MIDDLEBAND', 'RSI6', 'RSI12', 'RSI24', 'NATR', 'CDL3OUTSIDE', 'TRANGE', 'CDLBELTHOLD', 'CDLCLOSINGMARUBOZU']

# 选择全部特征或者筛选特征
features = selected_features

x_train = train_df[features].values
y_train = train_df['label'].values
x_valid = valid_df[features].values
y_valid = valid_df['label'].values
x_test = test_df[features].values
y_test = test_df['label'].values
x_train

array([[3296.258 , 3538.689 , 3295.741 , ...,  243.441 , -100.    ,
        -100.    ],
       [3287.711 , 3328.139 , 3189.605 , ...,  138.534 ,  100.    ,
           0.    ],
       [3361.84  , 3362.974 , 3288.933 , ...,   75.263 ,  100.    ,
         100.    ],
       ...,
       [3369.7334, 3371.4537, 3344.2822, ...,   27.1715,    0.    ,
           0.    ],
       [3408.3071, 3408.3071, 3364.4919, ...,   43.8152,    0.    ,
         100.    ],
       [3391.7551, 3456.7365, 3391.7551, ...,   64.9814,    0.    ,
        -100.    ]])

In [6]:
def model_report(y_true, y_pred, name=''):
    print(f'模型{name}')
    print('指标报告:')
    print(classification_report(y_true, y_pred))
    print(f'准确率: {accuracy_score(y_true, y_pred)}')


# 多模型训练

In [7]:
def print_true_pred(pred_values):
    pred_df = pd.DataFrame()
    pred_df['date'] = test_df.index.get_level_values(0).values
    pred_df['true'] = test_df['label'].values
    pred_df['pred'] = pred_values
    return pred_df

In [8]:
# logistic
from sklearn.linear_model import LogisticRegression
lrModel = LogisticRegression(penalty='l2')
lrModel.fit(x_train, y_train)
pred_lrModel = lrModel.predict(x_test)

model_report(y_test, pred_lrModel, 'logistic')
print_true_pred(pred_lrModel)

模型logistic
指标报告:
              precision    recall  f1-score   support

           0       0.00      0.00      0.00        17
           1       0.51      1.00      0.68        18

    accuracy                           0.51        35
   macro avg       0.26      0.50      0.34        35
weighted avg       0.26      0.51      0.35        35

准确率: 0.5142857142857142


,date,true,pred
0,2020-12-01,0,1
1,2020-12-02,0,1
2,2020-12-03,1,1
3,2020-12-04,0,1
4,2020-12-07,0,1
5,2020-12-08,0,1
6,2020-12-09,1,1
7,2020-12-10,0,1
8,2020-12-11,1,1
9,2020-12-14,0,1


In [9]:
# GBDT
from sklearn.ensemble import GradientBoostingClassifier
gbdtModel = GradientBoostingClassifier(n_estimators=200)
gbdtModel.fit(x_train, y_train)
pred_gbdtModel = gbdtModel.predict(x_test)

model_report(y_test, pred_gbdtModel, 'GBDT')
print_true_pred(pred_gbdtModel)

模型GBDT
指标报告:
              precision    recall  f1-score   support

           0       0.53      0.59      0.56        17
           1       0.56      0.50      0.53        18

    accuracy                           0.54        35
   macro avg       0.54      0.54      0.54        35
weighted avg       0.54      0.54      0.54        35

准确率: 0.5428571428571428


,date,true,pred
0,2020-12-01,0,0
1,2020-12-02,0,0
2,2020-12-03,1,0
3,2020-12-04,0,0
4,2020-12-07,0,0
5,2020-12-08,0,1
6,2020-12-09,1,0
7,2020-12-10,0,1
8,2020-12-11,1,1
9,2020-12-14,0,1


In [10]:
# SVC
from sklearn.svm import SVC
svmModel = SVC(kernel='rbf', probability=True)
svmModel.fit(x_train, y_train)
pred_svmModel = svmModel.predict(x_test)

model_report(y_test, pred_svmModel, 'SVC')
print_true_pred(pred_svmModel)

模型SVC
指标报告:
              precision    recall  f1-score   support

           0       0.00      0.00      0.00        17
           1       0.51      1.00      0.68        18

    accuracy                           0.51        35
   macro avg       0.26      0.50      0.34        35
weighted avg       0.26      0.51      0.35        35

准确率: 0.5142857142857142


,date,true,pred
0,2020-12-01,0,1
1,2020-12-02,0,1
2,2020-12-03,1,1
3,2020-12-04,0,1
4,2020-12-07,0,1
5,2020-12-08,0,1
6,2020-12-09,1,1
7,2020-12-10,0,1
8,2020-12-11,1,1
9,2020-12-14,0,1


In [11]:
# QDA
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
qdaModel = QuadraticDiscriminantAnalysis()
qdaModel.fit(x_train, y_train)
pred_qdaModel = qdaModel.predict(x_test)
model_report(y_test, pred_qdaModel, 'QuadraticDiscriminantAnalysis')
print_true_pred(pred_qdaModel)

模型QuadraticDiscriminantAnalysis
指标报告:
              precision    recall  f1-score   support

           0       0.75      0.71      0.73        17
           1       0.74      0.78      0.76        18

    accuracy                           0.74        35
   macro avg       0.74      0.74      0.74        35
weighted avg       0.74      0.74      0.74        35

准确率: 0.7428571428571429


,date,true,pred
0,2020-12-01,0,0
1,2020-12-02,0,0
2,2020-12-03,1,0
3,2020-12-04,0,0
4,2020-12-07,0,0
5,2020-12-08,0,0
6,2020-12-09,1,0
7,2020-12-10,0,0
8,2020-12-11,1,1
9,2020-12-14,0,0


In [69]:
# RandomForest
from sklearn.ensemble import RandomForestClassifier
rfModel = RandomForestClassifier(n_estimators=10000, random_state=20)
rfModel.fit(x_train, y_train)
pred_rfModel = rfModel.predict(x_test)

model_report(y_test, pred_rfModel, 'RandomForest')
print_true_pred(pred_rfModel)

模型RandomForest
指标报告:
              precision    recall  f1-score   support

           0       0.57      0.67      0.62        18
           1       0.57      0.47      0.52        17

    accuracy                           0.57        35
   macro avg       0.57      0.57      0.57        35
weighted avg       0.57      0.57      0.57        35

准确率: 0.5714285714285714


,date,true,pred
0,2020-12-01,0,0
1,2020-12-02,0,0
2,2020-12-03,1,0
3,2020-12-04,0,0
4,2020-12-07,0,0
5,2020-12-08,0,1
6,2020-12-09,1,1
7,2020-12-10,0,1
8,2020-12-11,1,1
9,2020-12-14,0,1


In [70]:
# 特征重要性
importances = rfModel.feature_importances_
features_importances = [[feature, importance] for feature, importance in zip(features, importances)]
features_importances_df = pd.DataFrame(features_importances, columns=['feature', 'importance'])
features_importances_df = features_importances_df.sort_values(by='importance', ascending=False)
features_sorted = features_importances_df['feature'].values
features_importances_df

,feature,importance
19,D,0.045406
27,NATR,0.043731
12,ULTOSC,0.043329
29,TRANGE,0.042879
7,amount,0.042116
6,pct_chg,0.041931
18,K,0.041841
5,change,0.040883
8,ADX,0.039015
20,DEM,0.038403


# 融合模型

In [71]:
# 融合模型
from sklearn.ensemble import VotingClassifier
clf1 = QuadraticDiscriminantAnalysis()
clf2 = RandomForestClassifier(n_estimators=1000, random_state=20)
clf3 = GradientBoostingClassifier(n_estimators=200)

eclf = VotingClassifier(estimators=[('QDA', clf1), ('RF', clf2), ('GBDT', clf3)], voting='soft')
eclf.fit(x_train, y_train)


VotingClassifier(estimators=[('QDA', QuadraticDiscriminantAnalysis()),
                             ('RF',
                              RandomForestClassifier(n_estimators=1000,
                                                     random_state=20)),
                             ('GBDT',
                              GradientBoostingClassifier(n_estimators=200))],
                 voting='soft')

In [72]:
# 预测
pred_eclf = eclf.predict(x_test)
model_report(y_test, pred_eclf, 'eclf')
print_true_pred(pred_eclf)

模型eclf
指标报告:
              precision    recall  f1-score   support

           0       0.59      0.56      0.57        18
           1       0.56      0.59      0.57        17

    accuracy                           0.57        35
   macro avg       0.57      0.57      0.57        35
weighted avg       0.57      0.57      0.57        35

准确率: 0.5714285714285714


,date,true,pred
0,2020-12-01,0,0
1,2020-12-02,0,0
2,2020-12-03,1,0
3,2020-12-04,0,0
4,2020-12-07,0,0
5,2020-12-08,0,1
6,2020-12-09,1,0
7,2020-12-10,0,0
8,2020-12-11,1,1
9,2020-12-14,0,1
